UCEC Reccurance Notebook

Importing data from ucec_tcga_pan_can_atlas_2018

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

df = pd.read_csv("ucec_tcga_pan_can_atlas_2018\data_clinical_patient.txt", sep="\t", comment="#", low_memory=False)

Removing unecessary columns from clinical data.

In [15]:
MAX_NULL_VALS = 0.3

og_columns = df.columns

for col in df.columns:
    # print(f"number of unique values in {col}: {len(df[col].unique())}")  # Show only first 5 unique values
    # print("-" * 50)
    # if there is only one value for every patient, remove the column
    if len(df[col].dropna().unique()) == 1:
        df.drop([col], axis=1, inplace=True)

# remove the column is over MAX_NULL_VALS percent null values
df.dropna(axis=1, thresh=len(df) * (1 - MAX_NULL_VALS))

pair_counts = df.groupby(['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT', 'DFS_STATUS']).size().reset_index(name='Count')

# Print the pairings and the count
print(pair_counts)

# remove non-informational columns
df = df.drop(columns=['PATIENT_ID', 'OTHER_PATIENT_ID'])

# remove non relevent data and data that was taken after inital recovery
to_be_removed = ["FORM_COMPLETION_DATE", 
                 ]

  NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT             DFS_STATUS  Count
0                                      No          0:DiseaseFree    328
1                                      No  1:Recurred/Progressed     12
2                                     Yes          0:DiseaseFree      7
3                                     Yes  1:Recurred/Progressed     38


Transforms data by changing catagorical data into numerical data and filling in missing data points with medians or modes.

In [27]:
#df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

categorical_columns_past_gen_liked = ["HISTORY_NEOADJUVANT_TRTYN",
                        "ETHNICITY",
                        "HISTORY_NEOADJUVANT_TRTYN",
                        "ICD_10", 
                        "PRIOR_DX", 
                        "RACE",
                        "RADIATION_THERAPY", 
                        "IN_PANCANPATHWAYS_FREEZE", 
                        "PFS_STATUS", 
                        "GENETIC_ANCESTRY_LABEL"] # do further research on what ICD_10 and ICD_O_3_SITE are



# Fill numerical NaNs with median
numerical_df = df.select_dtypes(include=['number'])
numerical_df = numerical_df.fillna(numerical_df.median())

# fill catagorical columns with mode
categorical_df = df.select_dtypes(exclude=['number'])
print(categorical_df.columns)
print("categorical_cols columns:", categorical_df)
categorical_df = categorical_df.fillna(categorical_df.mode())

# One-Hot Encode categorical columns (drop first to avoid redundancy)
categorical_df = pd.get_dummies(categorical_df, drop_first=True, dtype=float)
print(categorical_df)
# comment, right now I'm making eveery column be numerical, may change some to boolean if that would also work



Index(['SUBTYPE', 'DAYS_LAST_FOLLOWUP', 'ETHNICITY', 'FORM_COMPLETION_DATE',
       'ICD_10', 'ICD_O_3_HISTOLOGY', 'ICD_O_3_SITE',
       'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT',
       'PERSON_NEOPLASM_CANCER_STATUS', 'PRIOR_DX', 'RACE',
       'RADIATION_THERAPY', 'IN_PANCANPATHWAYS_FREEZE', 'OS_STATUS',
       'DSS_STATUS', 'DFS_STATUS', 'PFS_STATUS', 'GENETIC_ANCESTRY_LABEL'],
      dtype='object')
categorical_cols columns:           SUBTYPE DAYS_LAST_FOLLOWUP               ETHNICITY  \
0    UCEC_CN_HIGH               1249  Not Hispanic Or Latino   
1     UCEC_CN_LOW                 13  Not Hispanic Or Latino   
2    UCEC_CN_HIGH               1202  Not Hispanic Or Latino   
3     UCEC_CN_LOW                313  Not Hispanic Or Latino   
4       UCEC_POLE                 17  Not Hispanic Or Latino   
..            ...                ...                     ...   
524  UCEC_CN_HIGH                125  Not Hispanic Or Latino   
525      UCEC_MSI                631  Not Hispanic Or 